In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf

from pandas_datareader.data import DataReader

In [ ]:
from matplotlib import rcParams

# Adjust tick placement
rcParams['xtick.direction'] = 'in'
rcParams['ytick.direction'] = 'in'
rcParams['xtick.top'] = True
rcParams['ytick.right'] = True

# Disable legend frame
rcParams['legend.frameon'] = False

In [ ]:
start = '1945-01'
end   = '2045-01'

In [ ]:
pop = DataReader(['B230RC0Q173SBEA', 'CNP16OV', 'CLF16OV'], 'fred', start=start, end=end)

pop = pop.resample('QS').mean()

In [ ]:
pop_c = pd.DataFrame()
pop_t = pd.DataFrame()

for col in pop.columns:
    pop_c[col], pop_t[col] = sm.tsa.filters.hpfilter((np.log(pop[col].dropna())))
    
pop_smooth = np.exp(pop_t)

In [ ]:
rec = DataReader('USREC', 'fred', start=start, end=end)
rec = rec.dropna()

In [ ]:
# https://fred.stlouisfed.org/release/tables?rid=53

gdp = DataReader(['GDP', 'PCEC', 'PCDG', 'PCND', 'PCESV', 'GPDI', 'FPI', 'CBI', 'GCE', 'NETEXP', 'GDPDEF', 
                  'GDI', 'GDICOMP', 'GDINOS', 'PROPINC', 'COFC', 'GDITAXES', 'GDISUBS', 'A030RC1Q027SBEA'], 
                  'fred', start=start, end=end)

In [ ]:
GDINOS = gdp['GDP'] - gdp['A030RC1Q027SBEA'] - gdp['GDICOMP'] - gdp['GDITAXES'] + gdp['GDISUBS'] - gdp['COFC']
gdp['GDINOS'] = GDINOS

In [ ]:
# https://fred.stlouisfed.org/release/tables?rid=47&eid=155136

# , 'PRS85006013', 'PAYEMS'

nfb = DataReader(['HOANBS', 'PRS85006053', 'B4701C0A222NBEA', 'PRS85006023'], 
                    'fred', start=start, end=end)

In [ ]:
hours = pd.DataFrame()

hours['Hours'] = (nfb['HOANBS'] / 100 * nfb['B4701C0A222NBEA']['2012-01-01'] / 4
                  * gdp['GDP'] / (gdp['GDP']['2012'].mean()) / nfb['PRS85006053'] * 100).resample('QS').mean()


# hours['Employment'] = nfb['PAYEMS'].resample('QS').mean()
# hours['Hours per Employee'] = hours['Hours'] / hours['Employment']

hours['Hours per Employee'] = nfb['PRS85006023']

hours['Employment'] = hours['Hours'] / hours['Hours per Employee']

hours = hours['1948':]

# hours

In [ ]:
h_hp_cycles = pd.DataFrame()
h_hp_trend = pd.DataFrame()

h_cf_cycles = pd.DataFrame()
h_cf_trend = pd.DataFrame()

for col in hours.columns:
    h_hp_cycles[col], h_hp_trend[col] = sm.tsa.filters.hpfilter((100*np.log(hours[col].dropna())), lamb=1600)
    h_cf_cycles[col], h_cf_trend[col] = sm.tsa.filters.cffilter((100*np.log(hours[col].dropna())), low=6, high=32)

In [ ]:
h_cf_cycles[['Hours', 'Employment']].to_period('D').plot(color=['C0', 'C3'], lw=2)

plt.ylim(-6, 6)
plt.legend(ncol=2, loc='lower center')
plt.xlabel('')

plt.show()

h_cf_cycles[['Hours', 'Hours per Employee']].to_period('D').plot(color=['C0', 'C2'], lw=2)

plt.ylim(-6, 6)
plt.legend(ncol=2, loc='lower center')
plt.xlabel('')

plt.show()

h_hp_cycles.cov()

In [ ]:
# Estimate Capital series using PIM

cap_estim = pd.DataFrame()

cap_estim['Inv'] = (gdp['PCDG']+gdp['FPI'])/gdp['GDPDEF']*100/4
cap_estim['LnInv'] = np.log(cap_estim['Inv'])
cap_estim['t'] = np.arange(len(cap_estim['Inv']))

trend = smf.ols(formula='LnInv ~ t', data=cap_estim).fit()
intercept, slope = trend.params

delta = 0.017

K = np.zeros(len(cap_estim['LnInv']))
K_init = np.exp(intercept-slope)/(slope+delta)
K[0] = (1-delta)*K_init+cap_estim['Inv'][0]
for i in range(1,len(cap_estim['Inv'])):
    K[i] = (1-delta)*K[i-1]+cap_estim['Inv'][i]
cap_estim['K'] = K * 4 # annualization

cap_estim['K'].to_period('D').plot(lw=2)
plt.show()

(cap_estim['K']/(gdp['GDP']/gdp['GDPDEF']*100)).to_period('D').plot(lw=2)
plt.show()

In [ ]:
fig, ax = plt.subplots()

yy = (gdp['GDP']*1e9)/(gdp['GDPDEF']/1e2)/(pop_smooth['B230RC0Q173SBEA']*1e3)

np.log(yy).to_period('D').plot(ax=ax, lw=2, style='k-')

ticks = [10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000]
ax.set_yticks(np.log(ticks))
ax.set_yticklabels(ticks)

ax.set_ylim(np.log(ticks[0]), np.log(ticks[-1]))

ylim = ax.get_ylim()
ax.set_ylim(ylim)

ax.fill_between(rec.index, ylim[0], ylim[1], rec['USREC'], facecolor='lightgrey', edgecolor='lightgrey')

plt.title('US real per capita GDP (2017 $), log scale')
plt.xlabel('')
plt.show()

In [ ]:
dta = pd.DataFrame()

dta['Output'] = gdp['GDP']*10**9 / (gdp['GDPDEF']/100) / (pop_smooth['CNP16OV']*10**3)

dta['Consumption'] = (gdp['PCND']+gdp['PCESV'])*10**9 / (gdp['GDPDEF']/100) / (pop_smooth['CNP16OV']*10**3)

dta['Investment'] = (gdp['PCDG']+gdp['FPI'])*10**9 / (gdp['GDPDEF']/100) / (pop_smooth['CNP16OV']*10**3)

dta['Government'] = (gdp['GCE']+gdp['NETEXP']+gdp['CBI'])*10**9 / (gdp['GDPDEF']/100) / (pop_smooth['CNP16OV']*10**3)

dta['Capital'] = cap_estim['K'] * 1e9 / (pop_smooth['CNP16OV']*10**3)

dta['Hours'] = hours['Hours']*10**6 / (pop_smooth['CNP16OV']*10**3)

dta['Wages'] = ( (gdp['GDICOMP']+2/3*gdp['PROPINC'])*10**9 / (gdp['GDPDEF']/100) / (pop_smooth['CNP16OV']*10**3)
                / (dta['Hours']) )

dta['Return'] = 400 * ( ( (gdp['GDINOS'] - 2/3*gdp['PROPINC'])*1e9 ) 
                       / (gdp['GDPDEF']/100) / (cap_estim['K'] * 1e9))

α = 0.355 #1/3
dta['TFP'] = dta['Output'] / dta['Capital']**α / dta['Hours']**(1-α)

# dta = dta.dropna()

dta = dta['1948':]
dta = dta.dropna(how='all')

dta.head()

In [ ]:
dta.tail()

In [ ]:
# Calibration targets

print()
print('Shares in GDP')
print( 'C/Y  = ', (dta['Consumption']/dta['Output']).mean() )
print( 'I/Y  = ', (dta['Investment']/dta['Output']).mean() )
print( 'G/Y  = ', (dta['Government']/dta['Output']).mean() )
print( 'K/Y  =', (dta['Capital']/dta['Output']).mean() )
print()
print('Statistics for GDP / adult')
print( 'n_ss = ', 400*np.log(pop_smooth['CNP16OV']).diff().mean() )
print( 'x_ss = ', 400*np.log(dta['Output']).diff().mean() )
print()
print('Alternative GDP / capita')
print( 'n_ss = ', 400*np.log(pop_smooth['B230RC0Q173SBEA']).diff().mean() )
print( 'x_ss = ', 400*np.log(gdp['GDP']*10**9 / (gdp['GDPDEF']/100) / (pop_smooth['B230RC0Q173SBEA']*10**3)).diff().mean() )
print()

In [ ]:
out = pd.DataFrame()

out['y_obs'] = np.log(dta['Output']) #/dta['Output'].iloc[0]
out['dy_obs'] = 100*np.log(dta['Output']/dta['Output'].shift())
out['dy_obs_demean'] = out['dy_obs']-out['dy_obs'].mean()
out['g_y_obs'] = dta['Government'] / dta['Output']

In [ ]:
out.dropna(how='all').to_excel('Dane/rbc_simple_data.xlsx')

In [ ]:
tfp = pd.DataFrame()

tfp['TFP'] = np.log(dta['TFP'])
tfp['t'] = np.arange(len(dta['TFP']))

trend_TFP = smf.ols(formula='TFP ~ t', data=tfp).fit()
tfp['TFP_resid'] = trend_TFP.resid
intercept_TFP, slope_TFP = trend_TFP.params
print(trend_TFP.summary())

In [ ]:
yyy = intercept_TFP + slope_TFP*tfp['t']
yyy.plot(color='C3', lw=2)
tfp['TFP'].to_period('D').plot(color='C0', lw=2)
plt.fill_between(tfp.index, tfp['TFP'], yyy, color='C2', alpha=0.5)
plt.xlabel('')
plt.show()

In [ ]:
plt.hlines(0, tfp.index[0], tfp.index[-1], color='C3', lw=1)
(100*tfp['TFP_resid']).to_period('D').plot(color='C2', lw=2, label='$100 \cdot \ln z$')
plt.xlabel('')
plt.legend()
plt.show()

In [ ]:
resid_TFP = smf.ols(formula='TFP_resid ~ TFP_resid.shift() -1', data=tfp).fit()
print(resid_TFP.summary())

In [ ]:
print(resid_TFP.params[0])
print(100*resid_TFP.resid.std())

In [ ]:
from scipy.stats import norm

data = 100*resid_TFP.resid

mu, std = norm.fit(data) 

plt.hist(data, bins=21, density=True, alpha=0.6)

xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
  
plt.plot(x, p, 'C3', linewidth=2)

plt.show()

In [ ]:
hp_cycles = pd.DataFrame()
hp_trend = pd.DataFrame()

cf_cycles = pd.DataFrame()
cf_trend = pd.DataFrame()

for col in dta.columns:
    hp_cycles[col], hp_trend[col] = sm.tsa.filters.hpfilter((100*np.log(dta[col])).dropna(), lamb=1600)
    cf_cycles[col], cf_trend[col] = sm.tsa.filters.cffilter((100*np.log(dta[col])).dropna(), low=6, high=32)

In [ ]:
hp_cycles['Output'].to_period('D').plot(lw=2, label='Hodrick-Prescott') #color='k', 
cf_cycles['Output'].to_period('D').plot(lw=2, label='Christiano-Fitzgerald') #color='C3', 

plt.xlabel('')
# plt.legend(loc='lower left', ncol=2)
plt.legend(loc='upper right', ncol=2)

plt.ylim(-6, 6)

plt.show()

hp_cycles['Output'].corr(cf_cycles['Output'])

In [ ]:
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6), 
      (ax7, ax8, ax9)) = plt.subplots(3, 3, figsize=(16,9), sharex=False, sharey=False)

cf_cycles[['Output']].to_period('D').plot(ax=ax1, ylim=[-6,6], lw=2)

cf_cycles[['Output','Consumption']].to_period('D').plot(ax=ax2, ylim=[-6,6], lw=2)

cf_cycles[['Output','Investment']].to_period('D').plot(ax=ax3, ylim=[-15,15], lw=2)

cf_cycles[['Output','Government']].to_period('D').plot(ax=ax4, ylim=[-10,10], lw=2)

cf_cycles[['Output','Hours']].to_period('D').plot(ax=ax5, ylim=[-6,6], lw=2)

cf_cycles[['Output','Capital']].to_period('D').plot(ax=ax6, ylim=[-6,6], lw=2)

cf_cycles[['Output','TFP']].to_period('D').plot(ax=ax7, ylim=[-6,6], lw=2)

cf_cycles[['Output','Wages']].to_period('D').plot(ax=ax8, ylim=[-6,6], lw=2)

cf_cycles[['Output','Return']].to_period('D').plot(ax=ax9, ylim=[-15,15], lw=2)

ylim = ax1.get_ylim()
ax1.fill_between(rec.index, ylim[0], ylim[1], rec['USREC'], facecolor='lightgrey', edgecolor='lightgrey')

l = ax1.legend(ncol=2, frameon=True, loc='upper right')
l.get_frame().set_alpha(1)

ax2.legend(frameon=False, ncol=2, loc='lower center')
ax3.legend(frameon=False, ncol=2, loc='lower center')
ax4.legend(frameon=False, ncol=2, loc='lower center')
ax5.legend(frameon=False, ncol=2, loc='lower center')
ax6.legend(frameon=False, ncol=2, loc='lower center')
ax7.legend(frameon=False, ncol=2, loc='lower center')
ax8.legend(frameon=False, ncol=2, loc='lower center')
ax9.legend(frameon=False, ncol=2, loc='lower center')

ax1.set_xlabel('')
ax2.set_xlabel('')
ax3.set_xlabel('')
ax4.set_xlabel('')
ax5.set_xlabel('')
ax6.set_xlabel('')
ax7.set_xlabel('')
ax8.set_xlabel('')
ax9.set_xlabel('')

plt.show()

In [ ]:
fig, ax = plt.subplots()

cf_cycles['Investment / 3'] = cf_cycles['Investment'] / 3
cf_cycles[['Output','Consumption','Investment / 3','Hours']].to_period('D').plot(ax=ax, lw=2, ylim=[-8,6])

ylim = ax.get_ylim()

ax.fill_between(rec.index, ylim[0], ylim[1], rec['USREC'], facecolor='lightgrey', edgecolor='lightgrey')

l = plt.legend(ncol=2, frameon=True, loc='lower center')
l.get_frame().set_alpha(1)
        
plt.xlabel('')
plt.show()

In [ ]:
# # Run Matlab here
# break

In [ ]:
dyn_dta_2 = pd.read_csv('Dane/rbc_simple_g_out_2.csv')
dyn_dta_2.index = out.index[:304]	#hotfix

In [ ]:
dyn_hp_cycles_2 = pd.DataFrame()
dyn_hp_trend_2 = pd.DataFrame()

dyn_cf_cycles_2 = pd.DataFrame()
dyn_cf_trend_2 = pd.DataFrame()

for col in dyn_dta_2.columns:
    dyn_hp_cycles_2[col], dyn_hp_trend_2[col] = sm.tsa.filters.hpfilter(100*dyn_dta_2[col], lamb=1600)
    dyn_cf_cycles_2[col], dyn_cf_trend_2[col] = sm.tsa.filters.cffilter(100*dyn_dta_2[col], low=6, high=32)

In [ ]:
figs=(5,3)

for col in dyn_dta_2.columns[0:5]:
    plt.subplots(figsize=figs)
    np.log(dta[col]).to_period('D').plot(lw=2, label=col+' (d)')
    (dyn_dta_2[col]+np.log(dta['Output'].iloc[0])).to_period('D').plot(lw=2, label=col+' (m)')
    plt.xlabel('')
    plt.legend(frameon=False)
    plt.show()

In [ ]:
for col in ['Hours']:
    plt.subplots(figsize=figs)
    dta[col].to_period('D').plot(lw=2, label=col+' (d)')
    (np.exp(dyn_dta_2[col])*dta[col].mean()).to_period('D').plot(lw=2, label=col+' (m)')
    
    plt.xlabel('')
    plt.legend(frameon=False)
    plt.show()
    
for col in ['Wages']:
    plt.subplots(figsize=figs)
    
    (np.log(dta[col])-np.log(dta[col]).mean()).to_period('D').plot(lw=2, label=col+' (d)')
    (dyn_dta_2[col]-dyn_dta_2[col].mean()).to_period('D').plot(lw=2, label=col+' (m)')

    plt.xlabel('')
    plt.legend(frameon=False)
    plt.show()

In [ ]:
for col in ['Return']:
    plt.subplots(figsize=figs)
    dta[col].to_period('D').plot(lw=2, label=col+' (d)')
    (np.exp(dyn_dta_2[col])).to_period('D').plot(lw=2, label=col+' (m)')
    plt.xlabel('')
    plt.legend(frameon=False, ncol=2)
    plt.show()
    
    print(dta[col].mean())
    print(np.exp(dyn_dta_2[col]).mean())
    
for col in ['TFP']:
    plt.subplots(figsize=figs)
    (np.log(dta['TFP'])-np.log(dta['TFP']).mean()).plot(lw=2, label=col+' (d)')
    (dyn_dta_2['TFP']-dyn_dta_2['TFP'].mean()).to_period('D').plot(lw=2, label=col+' (m)')
    plt.xlabel('')
    plt.legend(frameon=False)
    plt.show()

In [ ]:
for col in dyn_dta_2.columns: #[:-2]:
    fig, ax = plt.subplots(figsize=figs)
    cf_cycles[col].to_period('D').plot(ax=ax, lw=2, label=col)
    dyn_cf_cycles_2[col].to_period('D').plot(ax=ax, lw=2, label='')
    
    yabs_max = abs(max(ax.get_ylim(), key=abs))
    ax.set_ylim(ymin=-yabs_max, ymax=yabs_max)
    
    plt.xlabel('')
    plt.legend(frameon=False, ncol=2) #, loc='upper center')
    plt.show()

In [ ]:
print('Standard Deviations')
print(hp_cycles['1960':'2015'].std())

print('')
print('Autocorrelations')
a = list(dta.columns.values)
for i in range(len(a)):
    print(dta.columns.values[i], '  \t', hp_cycles['1960':'2015'][hp_cycles.columns.values[i]].autocorr())

print('')
print('Correlations')
print(hp_cycles['1960':'2015'].corr())

In [ ]:
print('Standard Deviations')
print(dyn_hp_cycles_2['1960':'2015'].std())

print('')
print('Autocorrelations')
a = list(dyn_dta_2.columns.values)
for i in range(len(a)):
    print(dyn_dta_2.columns.values[i], '  \t', dyn_hp_cycles_2['1960':'2015'][dyn_hp_cycles_2.columns.values[i]].autocorr())

print('')
print('Correlations')
print(dyn_hp_cycles_2['1960':'2015'].corr())

In [ ]:
print(hp_cycles['1960':'2015'].corrwith(dyn_hp_cycles_2))

In [ ]:
h_cf_cycles['Hours per Employee'].to_period('D').plot(color='C2', lw=2, label='Hours per Employee (d)')
dyn_cf_cycles_2['Hours'].to_period('D').plot(color='C1', lw=2, label='Hours per Employee (m)')


plt.ylim(-6, 6)

plt.xlabel('')
plt.legend(frameon=False, ncol=2, loc='lower center') #, loc='upper center')
plt.show()

print(h_cf_cycles['Hours per Employee']['1960':'2015'].corr(dyn_cf_cycles_2['Hours']))